In [1]:
import pandas as pd
import sqlite3
import os

## 1. Extração e Carga (Extract & Load)

Carregamento dos arquivos CSV brutos diretamente para uma tabela de 'staging' no banco de dados.

In [2]:
if os.path.exists('turismo_dw_elt.db'):
    os.remove('turismo_dw_elt.db')

conn = sqlite3.connect('turismo_dw_elt.db')
cursor = conn.cursor()

files = [
    'dados/atendimentoturismo2022.csv',
    'dados/atendimentoturismo2023.csv',
    'dados/atendimentoturismo2024.csv'
]

for f in files:
    df = pd.read_csv(f, sep=';', encoding='utf-8')
    df.columns = [
        col.replace('tipohospedagem', 'tipo_hospedagem')
           .replace('tipotransporte', 'tipo_transporte')
           .replace('motivoviagem', 'motivo_viagem')
           .replace('tempoestadia', 'tempo_estadia')
           .replace('tipoatendimento', 'tipo_atendimento')
           .replace('municipointeresse', 'municipio_interesse')
           .replace('estadoorigem', 'estado_origem')
           .replace('paisorigem', 'pais_origem')
           .replace('qtdadoleslentes', 'qtd_adolescentes')
           .replace('qtdadultos', 'qtd_adultos')
           .replace('qtdcriancas', 'qtd_criancas')
           .replace('qtdidosos', 'qtd_idosos')
           .replace('qtdacompanhantes', 'qtd_acompanhantes')
           .replace('qtdturistas', 'qtd_turistas')
           .replace('faixaetaria', 'faixa_etaria')
           .replace('localatendimento', 'local_atendimento')
           .replace('idatendimento', 'id_atendimento')
           .replace('ehacompanhante', 'acompanhante')
    for col in df.columns
    ]
    df.to_sql('raw_atendimentos', conn, if_exists='append', index=False)
    print(f"Arquivo {f} carregado para staging. Linhas: {len(df)}")

Arquivo dados/atendimentoturismo2022.csv carregado para staging. Linhas: 37510
Arquivo dados/atendimentoturismo2023.csv carregado para staging. Linhas: 22239
Arquivo dados/atendimentoturismo2024.csv carregado para staging. Linhas: 596
Arquivo dados/atendimentoturismo2023.csv carregado para staging. Linhas: 22239
Arquivo dados/atendimentoturismo2024.csv carregado para staging. Linhas: 596


## 2. Transformação (Transform)

Utilizando SQL para limpar os dados e criar as tabelas de dimensão e fato.

### Limpeza de Dados
Removendo linhas que possuem valores nulos nas colunas que serão usadas nas dimensões (equivalente ao `dropna()` do ETL). As colunas de quantidade são ignoradas.

In [3]:
columns_to_check = [
    'ano', 'mes',
    'local_atendimento', 'municipio_interesse',
    'nacionalidade', 'pais_origem', 'estado_origem', 'cidade', 'sexo', 'faixa_etaria',
    'tipo_hospedagem', 'tipo_transporte', 'motivo_viagem', 'tempo_estadia', 'tipo_atendimento'
]

conditions = " OR ".join([f'"{col}" IS NULL' for col in columns_to_check])
query_clean = f"DELETE FROM raw_atendimentos WHERE {conditions}"

cursor.execute(query_clean)
conn.commit()

total = pd.read_sql("SELECT count(*) FROM raw_atendimentos", conn).iloc[0,0]
print(f"Limpeza concluída. Total de registros restantes: {total}")

Limpeza concluída. Total de registros restantes: 59160


### Criando Dimensão Tempo
Utilizando `LTRIM(RTRIM(...))` para simular o `strip()` do Python

In [4]:
cursor.execute("DROP TABLE IF EXISTS dim_tempo")
cursor.execute("""
CREATE TABLE dim_tempo (
    id_tempo INTEGER PRIMARY KEY AUTOINCREMENT,
    ano INTEGER,
    mes INTEGER
)
""")

cursor.execute("""
INSERT INTO dim_tempo (ano, mes)
SELECT DISTINCT 
    CAST(ano AS INTEGER), 
    CAST(mes AS INTEGER)
FROM raw_atendimentos
ORDER BY ano, mes
""")

pd.read_sql("SELECT * FROM dim_tempo LIMIT 5", conn)

,id_tempo,ano,mes
0,1,2022,1
1,2,2022,2
2,3,2022,3
3,4,2022,4
4,5,2022,5


### Criando Dimensão Local

In [5]:
cursor.execute("DROP TABLE IF EXISTS dim_local")
cursor.execute("""
CREATE TABLE dim_local (
    id_local INTEGER PRIMARY KEY AUTOINCREMENT,
    local_atendimento TEXT,
    municipio_interesse TEXT
)
""")

cursor.execute("""
INSERT INTO dim_local (local_atendimento, municipio_interesse)
SELECT DISTINCT 
    LTRIM(RTRIM(local_atendimento)), 
    LTRIM(RTRIM(municipio_interesse))
FROM raw_atendimentos
""")

pd.read_sql("SELECT * FROM dim_local LIMIT 5", conn)

,id_local,local_atendimento,municipio_interesse
0,1,CAT Praça do Arsenal,Nao Preencheu
1,2,CAT Praça de Boa Viagem,Nao Preencheu
2,3,CAT Praça de Boa Viagem,Recife
3,4,CAT Shopping Center Recife,Recife
4,5,CAT Shopping Center Recife,Nao Preencheu


### Criando Dimensão Perfil Turista

In [6]:
cursor.execute("DROP TABLE IF EXISTS dim_perfil_turista")
cursor.execute("""
CREATE TABLE dim_perfil_turista (
    id_perfil INTEGER PRIMARY KEY AUTOINCREMENT,
    nacionalidade TEXT,
    pais_origem TEXT,
    estado_origem TEXT,
    cidade TEXT,
    sexo TEXT,
    faixa_etaria TEXT
)
""")

cursor.execute("""
INSERT INTO dim_perfil_turista (nacionalidade, pais_origem, estado_origem, cidade, sexo, faixa_etaria)
SELECT DISTINCT 
    LTRIM(RTRIM(nacionalidade)),
    LTRIM(RTRIM(pais_origem)),
    LTRIM(RTRIM(estado_origem)),
    LTRIM(RTRIM(cidade)),
    LTRIM(RTRIM(sexo)),
    LTRIM(RTRIM(faixa_etaria))
FROM raw_atendimentos
""")

pd.read_sql("SELECT * FROM dim_perfil_turista LIMIT 5", conn)

,id_perfil,nacionalidade,pais_origem,estado_origem,cidade,sexo,faixa_etaria
0,1,Internacional,Franca,Nao Preencheu,,Fem,Adulto
1,2,Nacional,Brasil,Sao Paulo,,Masc,Adulto
2,3,Nacional,Brasil,Santa Catarina,,Fem,Adulto
3,4,Nacional,Brasil,Santa Catarina,,Masc,Adulto
4,5,Nacional,Brasil,Minas Gerais,,Fem,Adulto


### Criando Dimensão Detalhes Viagem

In [7]:
cursor.execute("DROP TABLE IF EXISTS dim_detalhes_viagem")
cursor.execute("""
CREATE TABLE dim_detalhes_viagem (
    id_detalhes INTEGER PRIMARY KEY AUTOINCREMENT,
    tipo_hospedagem TEXT,
    tipo_transporte TEXT,
    motivo_viagem TEXT,
    tempo_estadia TEXT,
    tipo_atendimento TEXT
)
""")

cursor.execute("""
INSERT INTO dim_detalhes_viagem (tipo_hospedagem, tipo_transporte, motivo_viagem, tempo_estadia, tipo_atendimento)
SELECT DISTINCT 
    LTRIM(RTRIM(tipo_hospedagem)),
    LTRIM(RTRIM(tipo_transporte)),
    LTRIM(RTRIM(motivo_viagem)),
    LTRIM(RTRIM(tempo_estadia)),
    LTRIM(RTRIM(tipo_atendimento))
FROM raw_atendimentos
""")

pd.read_sql("SELECT * FROM dim_detalhes_viagem LIMIT 5", conn)

,id_detalhes,tipo_hospedagem,tipo_transporte,motivo_viagem,tempo_estadia,tipo_atendimento
0,1,hotel,Aviao,,Outros,Presencial
1,2,hotel,Aviao,,5 a 6 dias,Presencial
2,3,hotel,Aviao,,1 a 2 dias,Presencial
3,4,outros,Nao Preencheu,,Outros,Presencial
4,5,Nao Preencheu,Onibus,,Nao Informou,Presencial


### Criando Tabela Fato
Relacionando a tabela bruta com as dimensões criadas para obter os IDs.

In [8]:
cursor.execute("DROP TABLE IF EXISTS fato_atendimentos")
cursor.execute("""
CREATE TABLE fato_atendimentos AS
SELECT 
    r.id_atendimento,
    t.id_tempo,
    l.id_local,
    p.id_perfil,
    d.id_detalhes
FROM raw_atendimentos r
LEFT JOIN dim_tempo t ON CAST(r.ano AS INTEGER) = t.ano AND CAST(r.mes AS INTEGER) = t.mes
LEFT JOIN dim_local l ON LTRIM(RTRIM(r.local_atendimento)) = l.local_atendimento 
                     AND LTRIM(RTRIM(r.municipio_interesse)) = l.municipio_interesse
LEFT JOIN dim_perfil_turista p ON LTRIM(RTRIM(r.nacionalidade)) = p.nacionalidade
                              AND LTRIM(RTRIM(r.pais_origem)) = p.pais_origem
                              AND LTRIM(RTRIM(r.estado_origem)) = p.estado_origem
                              AND LTRIM(RTRIM(r.cidade)) = p.cidade
                              AND LTRIM(RTRIM(r.sexo)) = p.sexo
                              AND LTRIM(RTRIM(r.faixa_etaria)) = p.faixa_etaria
LEFT JOIN dim_detalhes_viagem d ON LTRIM(RTRIM(r.tipo_hospedagem)) = d.tipo_hospedagem
                               AND LTRIM(RTRIM(r.tipo_transporte)) = d.tipo_transporte
                               AND LTRIM(RTRIM(r.motivo_viagem)) = d.motivo_viagem
                               AND LTRIM(RTRIM(r.tempo_estadia)) = d.tempo_estadia
                               AND LTRIM(RTRIM(r.tipo_atendimento)) = d.tipo_atendimento
""")

pd.read_sql("SELECT * FROM fato_atendimentos LIMIT 5", conn)

,id_atendimento,id_tempo,id_local,id_perfil,id_detalhes
0,151832,1,1,1,1
1,151833,1,1,2,2
2,151834,1,1,3,3
3,151835,1,1,4,3
4,151836,1,1,5,4


## 3. Verificação

Executando uma consulta de exemplo para validar o Data Warehouse.

In [9]:
query = """
SELECT 
    t.ano,
    l.municipio_interesse,
    COUNT(*) as total_atendimentos
FROM fato_atendimentos f
JOIN dim_tempo t ON f.id_tempo = t.id_tempo
JOIN dim_local l ON f.id_local = l.id_local
GROUP BY t.ano, l.municipio_interesse
ORDER BY t.ano, total_atendimentos DESC
LIMIT 10;
"""

pd.read_sql(query, conn)

,ano,municipio_interesse,total_atendimentos
0,2022,Nao Preencheu,20442
1,2022,Recife,11324
2,2022,Olinda,1156
3,2022,Jaboatão dos Guararapes,949
4,2022,Paulista,618
5,2022,Caruaru,348
6,2022,Camaragibe,211
7,2022,Cabo de Santo Agostinho,162
8,2022,Igarassu,133
9,2022,Vitória de Santo Antão,115


In [10]:
conn.close()